In [1]:
import os
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from pathlib import Path
import pandas as pd
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import mean_squared_error
from tensorflow.keras.backend import clear_session

2025-05-12 23:44:18.330690: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-12 23:44:18.338870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747107858.347913   92622 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747107858.350614   92622 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747107858.357567   92622 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Clear any previous session
clear_session()

In [3]:
# Import labels
labels_data = pd.read_csv('data/echonest_norm.csv').values
print(f"Label shape: {labels_data.shape}")

Label shape: (13131, 9)


In [4]:
def attach_label(image_path):
    try:
        image_id = int(image_path.split("/")[-1].split("_")[0])
        label = labels_data[labels_data[:, 0] == image_id, 1:]
        if label.shape[0] == 0:
            print(f"No label found for image {image_id}")
            return None
        return image_path, label.reshape(-1)
    except Exception as e:
        print(f"Error with {image_path}: {e}")
        return None

In [5]:
# Load image paths and labels
data_path = Path("spectrogram")
all_image_paths = sorted(map(str, data_path.glob("**/*.png")))
valid_pairs = list(filter(None, map(attach_label, all_image_paths)))

if len(valid_pairs) == 0:
    raise ValueError("No valid image-label pairs found!")

data_paths, labels = zip(*valid_pairs)

In [6]:
def load_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (int(984 / 3), int(2385 / 3)))
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image, label

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((list(data_paths), list(labels)))
dataset = dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle and split dataset
train_size = int(0.8 * len(data_paths))
train_dataset = dataset.take(train_size).batch(4).prefetch(tf.data.AUTOTUNE)
val_dataset = dataset.skip(train_size).batch(4).prefetch(tf.data.AUTOTUNE)

I0000 00:00:1747107861.212471   92622 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6


In [7]:
# Build model
clear_session()
base_model = ResNet50(weights=None, include_top=False, input_shape=(int(984 / 3), int(2385 / 3), 3))
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(8, activation='linear')
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 11, 25, 2048)   │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,114,312 (91.99 MB)

 Trainable params: 24,061,192 (91.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [8]:
try:
    with tf.device('/device:GPU:0'):
        model.fit(train_dataset, epochs=4, validation_data=val_dataset)
except Exception as e:
    print(f"Training failed: {e}")
    exit(1)

Epoch 1/4


I0000 00:00:1747107872.615592   92765 service.cc:152] XLA service 0x7fec8401feb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747107872.615617   92765 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2025-05-12 23:44:33.011095: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747107875.183489   92765 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-12 23:44:36.628570: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16390', 12 bytes spill stores, 12 bytes spill loads

2025-05-12 23:44:36.656855: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1639

2625/2625 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.2367 - mae: 0.2680

2025-05-12 23:49:38.487603: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1696', 12 bytes spill stores, 12 bytes spill loads



2625/2625 ━━━━━━━━━━━━━━━━━━━━ 336s 117ms/step - loss: 0.2367 - mae: 0.2679 - val_loss: 0.0803 - val_mae: 0.2170
Epoch 2/4
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 298s 113ms/step - loss: 0.0628 - mae: 0.1932 - val_loss: 0.2558 - val_mae: 0.2854
Epoch 3/4
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 299s 114ms/step - loss: 0.0621 - mae: 0.1910 - val_loss: 0.1086 - val_mae: 0.2315
Epoch 4/4
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 295s 112ms/step - loss: 0.0619 - mae: 0.1909 - val_loss: 0.1086 - val_mae: 0.2315


In [9]:
# Save results
y_test = []
y_pred = []
for images, batch_labels in tqdm(val_dataset):
    y_test.append(batch_labels.numpy())
    y_pred.append(model.predict(images, verbose=0))
y_test = np.concatenate(y_test, axis=0)
y_pred = np.concatenate(y_pred, axis=0)

# Create data frame with interleaved y_test and y_pred columns
data = {}
for i in range(y_test.shape[1]):
    data[f'y_test_{i}'] = y_test[:, i]
    data[f'y_pred_{i}'] = y_pred[:, i]
df = pd.DataFrame(data)
df.to_csv("data/evaluate.csv", index=False)

# Report MSE
mse = mean_squared_error(y_test, y_pred)
print(f"Mean squared error: {mse}")

100%|██████████| 657/657 [01:09<00:00,  9.45it/s]

Mean squared error: 0.10863168794079431


In [12]:
# Save the model
model.save("models/resnet50.keras")